In [78]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from google.colab import autoviz

# URL of the IPL Auction 2022 website
url = "https://www.iplt20.com/auction/2022"

# Send a GET request to the URL
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

In [79]:
# Find the table on the page based on its class or ID
# Adjust the class or ID according to the table you want to scrape
table = soup.find("table", class_="ih-td-tab auction-tbl")

In [80]:
import re
titles=[]
header_row = table.find_all("th")
for i in header_row:
    title=i.text
    #clean_title=re.sub("\n","",title)
    titles.append(title)

print(titles)

['TEAM', 'FUNDS REMAINING', 'OVERSEAS PLAYERS ', 'TOTAL PLAYERS']


In [81]:
rows=[]
#row = table.find_all("h2")
body_rows = table.find_all("tr")
for row in body_rows:
    row_data = [cell.text.strip() for cell in row.find_all("td")]
    rows.append(row_data)

print(rows)

[[], ['Chennai Super Kings', '₹2,95,00,000', '8', '25'], ['Delhi Capitals', '₹10,00,000', '7', '24'], ['Gujarat Titans', '₹15,00,000', '8', '23'], ['Kolkata Knight Riders', '₹45,00,000', '8', '25'], ['Lucknow Super Giants', '₹0', '7', '21'], ['Mumbai Indians', '₹10,00,000', '8', '25'], ['Punjab Kings', '₹3,45,00,000', '7', '25'], ['Rajasthan Royals', '₹95,00,000', '8', '24'], ['Royal Challengers Bangalore', '₹1,55,00,000', '8', '22'], ['Sunrisers Hyderabad', '₹10,00,000', '8', '23']]


In [82]:
df = pd.DataFrame(rows, columns=titles)

In [83]:
df.head(12)

,TEAM,FUNDS REMAINING,OVERSEAS PLAYERS,TOTAL PLAYERS
0,None,None,None,None
1,Chennai Super Kings,"₹2,95,00,000",8,25
2,Delhi Capitals,"₹10,00,000",7,24
3,Gujarat Titans,"₹15,00,000",8,23
4,Kolkata Knight Riders,"₹45,00,000",8,25
5,Lucknow Super Giants,₹0,7,21
6,Mumbai Indians,"₹10,00,000",8,25
7,Punjab Kings,"₹3,45,00,000",7,25
8,Rajasthan Royals,"₹95,00,000",8,24
9,Royal Challengers Bangalore,"₹1,55,00,000",8,22


In [84]:
df.drop(index=0, inplace=True)
df.head(10)

,TEAM,FUNDS REMAINING,OVERSEAS PLAYERS,TOTAL PLAYERS
1,Chennai Super Kings,"₹2,95,00,000",8,25
2,Delhi Capitals,"₹10,00,000",7,24
3,Gujarat Titans,"₹15,00,000",8,23
4,Kolkata Knight Riders,"₹45,00,000",8,25
5,Lucknow Super Giants,₹0,7,21
6,Mumbai Indians,"₹10,00,000",8,25
7,Punjab Kings,"₹3,45,00,000",7,25
8,Rajasthan Royals,"₹95,00,000",8,24
9,Royal Challengers Bangalore,"₹1,55,00,000",8,22
10,Sunrisers Hyderabad,"₹10,00,000",8,23


In [85]:
def heatmap(df, x_colname, y_colname, figscale=1, mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=(8 * figscale, 8 * figscale))
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(_df_16, *['OVERSEAS PLAYERS ', 'TOTAL PLAYERS'], **{})
chart

In [86]:
def categorical_histogram_sorted(df, colname, figscale=1, mpl_palette_name='Dark2'):
    from matplotlib import pyplot as plt
    import seaborn as sns

    # Calculate the counts for each category and sort them
    counts = df[colname].value_counts().sort_index()

    # Create the horizontal bar plot
    counts.plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))

    plt.gca().spines[['top', 'right']].set_visible(False)
    plt.xlabel('Count')
    plt.ylabel(colname)
    plt.title(f'{colname} DISTRIBUTION')

    return autoviz.MplChart.from_current_mpl_state()


chart = categorical_histogram_sorted(df, 'FUNDS REMAINING', figscale=1.5, mpl_palette_name='Set3')
chart


In [87]:
def categorical_histogram_sorted(df, colname, figscale=1, mpl_palette_name='Dark2'):
    from matplotlib import pyplot as plt
    import seaborn as sns

    # Calculate the counts for each category and sort them
    counts = df[colname].value_counts().sort_index()

    # Create the horizontal bar plot
    counts.plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))

    plt.gca().spines[['top', 'right']].set_visible(False)
    plt.xlabel('Count')
    plt.ylabel(colname)
    plt.title(f'{colname} Distribution')

    return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram_sorted(df, 'TOTAL PLAYERS', figscale=1.5, mpl_palette_name='Set3')
chart


In [88]:
def categorical_histogram(df, colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  df.groupby(colname).size().plot(kind='barh', color=sns.palettes.mpl_palette(mpl_palette_name), figsize=(8*figscale, 4.8*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  return autoviz.MplChart.from_current_mpl_state()

chart = categorical_histogram(_df_1, *['OVERSEAS PLAYERS '], **{})
chart